#Focus on the implementation of the query: search Publication by author's name input.
I start running the code with the merged sources: csv and json.

In [8]:
import pandas as pd
import json
from sqlalchemy import create_engine

class RelationalProcessor:
    def __init__(self, db_path):
        self.db_path = db_path
        self.engine = create_engine(f"sqlite:///{db_path}") 

    def upload_data(self, file_path):
        """
        Uploads data from a CSV or JSON file to the database.

        Args:
            file_path (str): Path to the file to be uploaded.

        Returns:
            bool: True if the upload was successful, False otherwise.
        """
        try:
            if file_path.endswith('.csv'):
                df = pd.read_csv(file_path, dtype={'publication_year': 'Int64'}) 
                df['internal_id'] = df.index  # Assign unique internal IDs

            elif file_path.endswith('.json'):
                with open(file_path, 'r') as f:
                    data = json.load(f)
                df = pd.json_normalize(data) 

            else:
                raise ValueError("Unsupported file type. Please provide a CSV or JSON file.")

            # Create tables in database (if not exist)
            df.to_sql('publications', self.engine, if_exists='replace', index=False) 

            return True
        except Exception as e:
            print(f"Error uploading data: {e}")
            return False

# Example usage
db_path = 'my_database.db' 
processor = RelationalProcessor(db_path)

csv_file_path = 'import/relational_publications.csv'
json_file_path = 'import/relational_other_data.json'

if processor.upload_data(csv_file_path):
    print("CSV data uploaded successfully.")

if processor.upload_data(json_file_path):
    print("JSON data uploaded successfully.")

CSV data uploaded successfully.
Error uploading data: (sqlite3.InterfaceError) Error binding parameter 0 - probably unsupported type.
[SQL: INSERT INTO publications ("authors.doi:10.1162/qss_a_00023", "authors.doi:10.1007/s11192-019-03217-6", "authors.doi:10.1007/s11192-019-03311-9", "authors.doi:10.1038/sdata.2016.18", "authors.doi:10.1371/journal.pbio.3000385", "authors.doi:10.3233/ds-190016", "authors.doi:10.1007/s11192-020-03397-6", "authors.doi:10.1186/s13321-020-00448-1", "authors.doi:10.1007/978-3-030-61244-3_16", "authors.doi:10.1007/978-3-030-61244-3_6", "authors.doi:10.1007/978-3-030-54956-5_2", "authors.doi:10.1007/978-3-030-55814-7_15", "authors.doi:10.1007/s11192-020-03690-4", "authors.doi:10.1007/978-3-030-62466-8_28", "authors.doi:10.1038/s41597-020-00749-y", "authors.doi:10.1007/978-3-030-77385-4_37", "authors.doi:10.1007/s11192-021-04079-7", "authors.doi:10.1162/qss_a_00112", "authors.doi:10.7717/peerj-cs.421", "authors.doi:10.1101/2021.05.04.442638", "authors.doi:10.1

## Data Model
Then I create the data model for the Query Processors, addind classes, relations and methods.

In [27]:
import pandas as pd
from pandas import read_csv
from pandas import read_json
from pandas import read_sql
from pandas import merge
from pandas import concat
from pandas import Series
import sqlite3
import json
from sqlite3 import connect


# class for Identifiable Entity
class IdentifiableEntity(object):

    def __init__(self, id):
        self.id = id
        self.id_array = set()
        for identifier in id:
            self.id_array.add(identifier)

    # methods of identifiableEntity

    def getIds(self):
        result = []
        for identifier in self.id_array:
            result.append(identifier)
        result.sort()
        return result

    def addId(self, id):
        result = True
        if id not in self.id:
            self.id.add(id)
        else:
            result = False
        return result

    def removeId(self, identifier):
        result = True
        if identifier in self.id:
            self.id.remove(identifier)
        else:
            result = False
        return result


# class for Person
class Person(IdentifiableEntity):

    def __init__(self, id, givenName, familyName, orcid):
        super().__init__(id)
        self.givenName = givenName
        self.familyName = familyName

    # methods of person
    def getGivenName(self):
        return self.givenName

    def getFamilyName(self):
        return self.familyName
    
    def getOrcid(self):
        return self.orcid


# class for publication
class Publication(IdentifiableEntity):
    def __init__(self, id, publicationYear, title, publicationVenue, author, cites):
        self.publicationYear = publicationYear
        self.title = title
        self.publicationVenue = publicationVenue
        self.author = set(author)
        self.cites = set(cites)

        super().__init__(id)

    # methods of publications
    def getPublicationYear(self):
        return self.publicationYear

    def getTitle(self):
        return self.title

    def getPublicationVenue(self):
        return self.publicationVenue

    def getCitedPublications(self):
        self.cites = []
        for p in self.cites:
            self.id.add(p)

        return self.cites

    def getAuthors(self):
        self.author = []
        for p in self.author:
            self.author.add(p)

        return self.author


# class for journal article
class JournalArticle(Publication):
    def __init__(self, id, publicationYear, title, publicationVenue, author, cites, issue, volume):
        self.issue = issue
        self.volume = volume
        super().__init__(id, publicationYear, title, publicationVenue, author, cites)

    # methods of journal article
    def getIssue(self):
        return self.issue

    def getVolume(self):
        return self.volume


# class for book chapter
class BookChapter(Publication):
    def __init__(self, id, publicationYear, title, publicationVenue, author, cites, chapterNumber):
        self.chapterNumber = chapterNumber
        super().__init__(id, publicationYear, title, publicationVenue, author, cites)

    def getChapterNumber(self):
        return self.chapterNumber

class ProceedingsPaper(Publication):
    pass


# class for venue
class Venue(IdentifiableEntity):
    def __init__(self, id, title, publisher):
        self.title = title
        self.publisher = set(publisher)
        super().__init__(id)

    def getTitle(self):
        return self.title

    def getPublisher(self):
        return self.publisher


class Journal(Venue):
    pass


class Book(Venue):
    pass


# class for proceedings
class Proceedings(Venue):
    def __init__(self, id, title, publisher, event):
        self.event = event
        super().__init__(id, title, publisher)

    def getEvent(self):
        return self.event


# class for organization
class Organization(IdentifiableEntity):
    def __init__(self, id, name):
        self.name = name
        super().__init__(id)

    def getName(self):
        return self.name


class QueryProcessor(object):
    def __init__(self):
        pass


# classes for the processors
class RelationalProcessor(object):

    def __init__(self):
        self.dbPath = ''  # dbPath: the variable containing the path of the database,
        # initially set as an empty string, that will be updated with the method setDbPath.

    # Methods
    def getDbPath(self):  # it returns the path of the database.
        return self.dbPath

    def setDbPath(self, path):
        self.dbPath = path


class RelationalDataProcessor(RelationalProcessor):

    def __init__(self):
        super().__init__()

    def uploadData(self, path): #method for uploading data. Here, in case the file is not a csv nor a json
        self.path = path        #an exception should be raised
        result = True

        while True:
            try:

                if self.path.endswith('csv'):
                    with open(path, "r", encoding="utf-8") as file:
                        publications = pd.read_csv(file, keep_default_na=False,
                                                   dtype={
                                                       "id": "string",
                                                       "title": "string",
                                                       "type": "string",
                                                       "publication_year": "int",
                                                       "issue": "string",
                                                       "volume": "string",
                                                       "chapter": "string",
                                                       "publication_venue": "string",
                                                       "venue_type": "string",
                                                       "publisher": "string",
                                                       "event": "string"
                                                   })
                        #creating empty data frames to be populated with the information coming from csv file
                        journal_article = pd.DataFrame({
                            "internalId", "issue", "volume", "publication_year", "title", "publication_venue", "id"})

                        book_chapter = pd.DataFrame({
                            "internalId", "chapter_number", "publication_year", "title", "publication_venue", "id"})

                        journal = pd.DataFrame({
                            "internalId", "doi", "title", "publisher"})

                        book = pd.DataFrame({
                            "internalId", "doi", "title", "publisher"})

                        proceedings_paper = pd.DataFrame(
                            {'internalId', 'doi', 'title', 'publication_year', 'publication_venue'})

                        proceedings = pd.DataFrame({
                            "internalId", "title", "event", "id", "publisher"})

                        publications = publications.drop_duplicates()

                        pub_ids = pd.DataFrame(publications['id'])
                        #creating internal Ids
                        publications_internal_id = []

                        for idx, row in pub_ids.iterrows():
                            publications_internal_id.append("publications-" + str(idx))

                        publications['internalId'] = pd.Series(publications_internal_id)

                        # Data Frame for publications

                        publications_df = pd.DataFrame(
                            {'internalId', 'doi', 'title', 'publication_year', 'publication_venue', 'publisher'})
                        publications_df['internalId'] = publications['internalId']
                        publications_df['doi'] = publications['id'].astype('str')
                        publications_df['title'] = publications['title'].astype('str')
                        publications_df['type'] = publications['type'].astype('str')
                        publications_df['publication_year'] = publications['publication_year'].astype('int')
                        publications_df['publication_venue'] = publications['publication_venue'].astype('str')
                        publications_df['publisher'] = publications['publisher'].astype('str')

                        # Data Frame for journal article

                        journal_article['internalId'] = publications[publications['type'] == "journal-article"][
                            'internalId'].astype('str')
                        journal_article['doi'] = publications[publications['type'] == "journal-article"]['id'].astype(
                            'str')
                        journal_article['issue'] = publications[publications['type'] == "journal-article"][
                            'issue'].astype('str')
                        journal_article['volume'] = publications[publications['type'] == "journal-article"][
                            'volume'].astype('str')
                        journal_article['publication_year'] = publications[publications['type'] == "journal-article"][
                            'publication_year'].astype('str')
                        journal_article['publication_venue'] = publications[publications['type'] == "journal-article"][
                            'publication_venue'].astype('str')
                        journal_article['title'] = publications[publications['type'] == "journal-article"][
                            'title'].astype('str')

                        # Data Frame for Journal

                        journal['internalId'] = publications[publications['venue_type'] == "journal"][
                            'internalId'].astype('str')
                        journal['doi'] = publications[publications['venue_type'] == "journal"]['id'].astype(
                            'str')
                        journal['title'] = publications[publications['venue_type'] == "journal"][
                            'issue'].astype('str')
                        journal['publisher'] = publications[publications['venue_type'] == "journal"][
                            'volume'].astype('str')

                        # Data Frame for book chapter

                        book_chapter['internalId'] = publications[publications['type'] == "book-chapter"][
                            'internalId'].astype('str')
                        book_chapter['doi'] = publications[publications['type'] == "book-chapter"]['id'].astype('str')
                        book_chapter['chapter'] = publications[publications['type'] == "book-chapter"][
                            'chapter'].astype('str')
                        book_chapter['publication_year'] = publications[publications['type'] == "book-chapter"][
                            'publication_year'].astype('str')
                        book_chapter['publication_venue'] = publications[publications['type'] == "book-chapter"][
                            'publication_venue'].astype('str')
                        book_chapter['title'] = publications[publications['type'] == "book-chapter"][
                            'title'].astype('str')

                        # Data Frame for book

                        book['internalId'] = publications[publications['venue_type'] == "book"][
                            'internalId'].astype('str')
                        book['doi'] = publications[publications['venue_type'] == "book"]['id'].astype(
                            'str')
                        book['title'] = publications[publications['venue_type'] == "book"][
                            'issue'].astype('str')
                        book['publisher'] = publications[publications['venue_type'] == "book"][
                            'volume'].astype('str')

                        # Data Frame for Proceedings Paper

                        proceedings_paper['internalId'] = publications['internalId'].astype('str')
                        proceedings_paper['doi'] = publications['id'].astype('str')
                        proceedings_paper['title'] = publications['title'].astype('str')
                        proceedings_paper['publisher'] = publications['publisher'].astype('str')
                        proceedings_paper['event'] = publications['event'].astype('str')
                        proceedings_paper['publication_venue'] = publications['publication_venue'].astype('str')
                        proceedings_paper['publication_year'] = publications['publication_year'].astype('str')

                        # Data Frame for Proceedings 

                        proceedings['internalId'] = publications['internalId'].astype('str')
                        proceedings['doi'] = publications['id'].astype('str')
                        proceedings['title'] = publications['title'].astype('str')
                        proceedings['publisher'] = publications['publisher'].astype('str')
                        proceedings['event'] = publications['event'].astype('str')
                        proceedings['publication_venue'] = publications['publication_venue'].astype('str')

                    with connect(self.dbPath) as con:
                        publications_df.to_sql("Publications", con, if_exists="append", index=False)
                        journal_article.to_sql("JournalArticle", con, if_exists="append", index=False)
                        book_chapter.to_sql("BookChapter", con, if_exists="append", index=False)
                        journal.to_sql("Journal", con, if_exists="append", index=False)
                        book.to_sql("Book", con, if_exists="append", index=False)
                        proceedings_paper.to_sql("ProceedingsPaper", con, if_exists="append", index=False)
                        proceedings.to_sql("Proceedings", con, if_exists="append", index=False)

                    con.commit()

                elif self.path.endswith('.json'):

                    with open(path, "r", encoding="utf-8") as file:
                        venue = json.load(file)

                        # DataFrame for authors being populated
                        authors_df = pd.DataFrame({
                            "doi_authors": pd.Series(dtype="str"),
                            "family": pd.Series(dtype="str"),
                            "given": pd.Series(dtype="str"),
                            "orcid": pd.Series(dtype="str")
                        })

                        family = []
                        given = []
                        orcid = []
                        doi_authors = []

                        authors = venue['authors']
                        for key in authors:
                            for value in authors[key]:
                                doi_authors.append(key)
                                family.append(value['family'])
                                given.append(value['given'])
                                orcid.append(value['orcid'])

                        authors_df['doi_authors'] = doi_authors
                        authors_df['family'] = family
                        authors_df['given'] = given
                        authors_df['orcid'] = orcid
                        authors_df = authors_df.drop_duplicates()

                        # Data Frame for internal ID Venue

                        venues_id_df = pd.DataFrame({
                            "doi_venues_id": pd.Series(dtype="str"),
                            "issn_isbn": pd.Series(dtype="str"),
                        })
                        doi_venues_id = []
                        issn_isbn = []

                        venues_id = venue["venues_id"]
                        for key in venues_id:
                            for value in venues_id[key]:
                                doi_venues_id.append(key)
                                issn_isbn.append(value)

                        venues_id_df["doi_venues_id"] = doi_venues_id
                        venues_id_df["issn_isbn"] = pd.Series(issn_isbn)

                        venues_id_df = venues_id_df.drop_duplicates()

                        venue_int = []
                        for idx, row in venues_id_df.iterrows():
                            venue_int.append("venue-" + str(idx))

                        venues_id_df["internalId"] = venue_int

                        # Data Frame for references
                        references_df = pd.DataFrame({
                            "idCited": pd.Series(dtype="str"),
                            "idCites": pd.Series(dtype="str"),
                            "no": pd.Series(dtype="int64")
                        })

                        id_ref = []
                        id_ref_doi = []

                        references = venue["references"]

                        for key in references:
                            for value in references[key]:
                                id_ref.append(key)
                                id_ref_doi.append(value)

                        references_df["idCited"] = id_ref
                        references_df["idCites"] = id_ref_doi
                        references_df["no"] = references_df.index

                        references_df = references_df.drop_duplicates()

                        # Data Frame for publishers

                        publishers_df = pd.DataFrame({
                            "id_pub": pd.Series(dtype="str"),
                            "name": pd.Series(dtype="str")
                        })

                        doi_pub = []
                        name = []
                        id_pub = []

                        publishers = venue["publishers"]

                        for key in publishers:
                            for value in publishers[key]:
                                doi_pub.append(key)
                                id_pub.append(publishers[key]["id"])
                                name.append(publishers[key]["name"])

                        publishers_df["doi"] = doi_pub
                        publishers_df["id_pub"] = id_pub
                        publishers_df["name"] = name
                        publishers_df["doi_venue"] = venues_id_df["doi_venues_id"]
                        publishers_df = publishers_df.drop_duplicates()

                        publishers_df["internalId"] = venues_id_df['internalId']

                        authors_df["internalId"] = venues_id_df['internalId']

                        references_df["internalId"] = venues_id_df['internalId']

                        # Creating the tables into the database

                    with connect(self.dbPath) as con:

                        authors_df.to_sql("Authors", con, if_exists="append", index=False)
                        venues_id_df.to_sql("Venues", con, if_exists="append", index=False)
                        references_df.to_sql("ReferencesTable", con, if_exists="append", index=False)
                        publishers_df.to_sql("Publishers", con, if_exists="append", index=False)

                    con.commit()

                else:
                    result = False

            except ValueError:
                print("Oops! This doesn't seem a valid file.")
                result = False

            return result
   



Now we can configure the two Query Processors and add the specific queries for the case:
Search Publication by Author's name.

In [29]:
class RelationalQueryProcessor:
    def __init__(self, db_path):
        self.db_path = db_path

    def create_tables(self):
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()

        # Create authors table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS authors (
                publication_id TEXT,
                first_name TEXT,
                last_name TEXT,
                ORCID TEXT,
                FOREIGN KEY (publication_id) REFERENCES publications(publication_id)
            )
        """)
        conn.commit()
        conn.close()

    def load_author_data_from_json(self, json_file_path):
        with open(json_file, 'r') as f:
            author_data = json.load(f)

        # Extract author information from JSON 
        author_list = []
        for doi, author_list in author_data.items(): 
            for author in author_list:
                author_list.append({
                    "publication_id": doi,
                    "first_name": author.get("given", ""),
                    "last_name": author.get("family", ""),
                    "ORCID": author.get("orcid", "") 
                })
        author_df = pd.DataFrame(author_list)
        
        # Load author data to database
        author_df.to_sql("authors", self.db_path, index=False, if_exists='append')

    def searchByAuthorName(self, name):
        conn = sqlite3.connect(self.db_path)
        cur = conn.cursor()
        query = """
        SELECT p.publication_id, p.title, a.first_name, a.last_name
        FROM publications p
        JOIN authors a ON p.publication_id = a.publication_id
        WHERE LOWER(a.first_name) LIKE ? OR LOWER(a.last_name) LIKE ?
        """
        cur.execute(query, (f"%{name.lower()}%", f"%{name.lower()}%"))
        results = cur.fetchall()
        conn.close()
        return pd.DataFrame(results, columns=["publication_id", "title", "first_name", "last_name"])
    

class TriplestoreQueryProcessor:
    def __init__(self):
        self.graph = Graph()

    def load_rdf(self, publication_data, author_data):
        """
        Load publication and author data into an RDF graph.
        """
        
    # Method:
    def uploadData(self, path): # it enables to upload the collection of data specified in the input file path (either in CSV or JSON) into the database.
        dsd_graph = Graph()
    store.open((endpoint, endpoint))
    for triple in dsd_graph.triples((None, None, None)):
        store.add(triple)
    store.close()
       

# classes of resources
    JournalArticle = URIRef("https://schema.org/ScholarlyArticle")
    BookChapter = URIRef("https://schema.org/Chapter")
    ProceedingsPaper = URIRef("http://purl.org/spar/fabio/ProceedingsPaper")
    Journal = URIRef("https://schema.org/Periodical")
    Book = URIRef("https://schema.org/Book")
    Proceedings = URIRef("http://purl.org/spar/fabio/AcademicProceedings")
    Organization = URIRef("https://schema.org/Organization")
    IdentifiableEntity = URIRef("https://schema.org/identifier")
    Publication = URIRef("https://schema.org/publication")
    Venue = URIRef("https://schema.org/VenueMap")
    Person = URIRef("https://schema.org/Person")

        # attributes related to classes
    publicationYear = URIRef("https://schema.org/datePublished")
    title = URIRef("http://purl.org/dc/terms/title")
    issue = URIRef("https://schema.org/issueNumber")
    volume = URIRef("https://schema.org/volumeNumber")
    doi = URIRef("https://schema.org/identifier")
    identifier = URIRef("https://schema.org/identifier")
    name = URIRef("https://schema.org/name")
    event = URIRef("https://schema.org/Event")
    chapterNumber = URIRef("https://github.com/lelax/D_Sign_Data/blob/main/URIRef/chapterNumber")
    givenName = URIRef("https://schema.org/givenName")
    familyName = URIRef("https://schema.org/familyName")
    orcid = URIRef("https://schema.org/orcid")
    proceedingpapers = URIRef("https://schema.org/proceedingpapers")
    doiPublisher = URIRef("https://schema.org/doiPublisher")
    doiId = URIRef("https://schema.org/doiId")

        # relations among classes
    publicationVenue = URIRef("https://schema.org/isPartOf")
    publisher = URIRef("https://schema.org/publishedBy")
    author = URIRef("http://purl.org/saws/ontology#isWrittenBy")
    citation = URIRef("https://schema.org/citation")

        #literal
    a_string = Literal("a string ")
    a_number = Literal(4)
    a_boolean = Literal(True)

    base_url = "https://github.com/lelax/New_Design_Data/"

    def searchByAuthorName(self, name):
        query = """
        SELECT ?publication ?publicationTitle
        WHERE {
            ?publication dc:title ?publicationTitle.
            ?publication dc:creator ?author.
            ?author foaf:firstName ?firstName.
            ?author foaf:familyName ?lastName.
            FILTER(
                REGEX(?firstName, "{name}", "i") || 
                REGEX(?lastName, "{name}", "i")
            )
        }
        """.format(name=name)
        results = self.graph.query(query)
        # Process results and return in desired format (e.g., DataFrame)
        return results 

# Example usage
# Load data from CSV and JSON (replace with your file paths)
publications_df = pd.read_csv("import/relational_publications.csv")
author_data = json.load(open("import/relational_other_data.json")) 

# Create and populate database (RelationalProcessor)
relational_processor = RelationalQueryProcessor("publications.db")  

# Create RDF graph (TriplestoreQueryProcessor)
rdf_processor = TriplestoreQueryProcessor()
rdf_processor.load_rdf(publications_df, author_data) 

# Search for publications by author name
author_name = "input here"
relational_results = relational_processor.searchByAuthorName(author_name)
rdf_results = rdf_processor.searchByAuthorName(author_name)

NameError: name 'store' is not defined